In [1]:
# import cv2
# import os
# import numpy as np

# def create_coordinate_system(image_path):
#     # Load the image
#     image = cv2.imread(image_path)

#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#     # Define the ArUco dictionary
#     aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)

#     # Initialize the ArUco detector parameters
#     parameters = cv2.aruco.DetectorParameters_create()
#     # parameters.adaptiveThreshWinSizeMin = 3
#     # parameters.adaptiveThreshWinSizeMax = 100

#     # Detect markers in the image
#     corners, ids, rejected = cv2.aruco.detectMarkers(blurred, aruco_dict, parameters=parameters)

#     # Draw the detected markers on the image
#     image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids)

#     # Create a coordinate system image
#     coordinate_system = np.zeros_like(image)

#     # Check if markers are detected
#     if ids is not None:
#         # Iterate over detected markers
#         for i in range(len(ids)):
#             # Draw a coordinate system for each marker
#             rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.1, camera_matrix, distortion_coefficients)
#             rvec = rvec[0]  # Extract the rotation vector

#             # Ensure rvec is a 1x3 vector
#             if rvec.shape == (1, 1, 3):
#                 rvec = rvec[0, 0]

#             # Print the coordinates of the marker
#             print(f"Marker ID: {ids[i]} - Translation Vector: {tvec[0]} - Rotation Vector: {rvec}")

#             # Draw the coordinate system
#             coordinate_system = cv2.aruco.drawAxis(
#                 coordinate_system, 
#                 np.float32(camera_matrix),  # Ensure camera_matrix is a 3x3 floating-point matrix
#                 np.float32(distortion_coefficients),  # Ensure distortion_coefficients is a 1x5 floating-point matrix
#                 rvec=rvec, 
#                 tvec=tvec[0], 
#                 length=0.1
#             )



#     # Save the images to files
#     cv2.imwrite('/home/qt/callib/imgs/Detected_Markers.jpg', image_with_markers)
#     cv2.imwrite('/home/qt/callib/imgs/Coordinate_System.jpg', coordinate_system)

# # Dummy camera matrix and distortion coefficients
# camera_matrix = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]], dtype=np.float32)
# distortion_coefficients = np.array([0.1, 0.01, 0, 0, 0], dtype=np.float32)

# # Path to the input image
# image_path = '/home/qt/callib/Align.jpg'

# # Call the function to create a coordinate system
# create_coordinate_system(image_path)

In [2]:
# import cv2
# import os
# import numpy as np

# def create_coordinate_system(image_path):
#     # Load the image
#     image = cv2.imread(image_path)

#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#     # Define the ArUco dictionary
#     aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)

#     # Initialize the ArUco detector parameters
#     parameters = cv2.aruco.DetectorParameters_create()

#     # Detect markers in the image
#     corners, ids, rejected = cv2.aruco.detectMarkers(blurred, aruco_dict, parameters=parameters)

#     # Draw the detected markers on the image
#     image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids)

#     # Create a coordinate system image
#     coordinate_system = np.zeros_like(image)

#     # Check if markers are detected
#     if ids is not None:
#         # Iterate over detected markers
#         for i in range(len(ids)):
#             # Draw a coordinate system for each marker
#             rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.1, camera_matrix, distortion_coefficients)
#             rvec = rvec[0]  # Extract the rotation vector

#             # Ensure rvec is a 1x3 vector
#             if rvec.shape == (1, 1, 3):
#                 rvec = rvec[0, 0]

#             # Print the coordinates of the marker
#             print(f"Marker ID: {ids[i]} - Translation Vector: {tvec[0]} - Rotation Vector: {rvec}")

#             # Draw the coordinate system
#             coordinate_system = cv2.aruco.drawAxis(
#                 coordinate_system, 
#                 np.float32(camera_matrix),  # Ensure camera_matrix is a 3x3 floating-point matrix
#                 np.float32(distortion_coefficients),  # Ensure distortion_coefficients is a 1x5 floating-point matrix
#                 rvec=rvec, 
#                 tvec=tvec[0], 
#                 length=0.05
#             )

#             # Draw a box around the coordinate system
#             box_size = 50  # Adjust the box size as needed
#             cv2.drawMarker(coordinate_system, (int(tvec[0, 0, 0]), int(tvec[0, 0, 1])), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=box_size, thickness=2)

#     # Save the images to files
#     cv2.imwrite('/home/qt/callib/imgs/Detected_Markers.jpg', image_with_markers)
#     cv2.imwrite('/home/qt/callib/imgs/Coordinate_System.jpg', coordinate_system)

# # Dummy camera matrix and distortion coefficients
# camera_matrix = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]], dtype=np.float32)
# distortion_coefficients = np.array([0.1, 0.01, 0, 0, 0], dtype=np.float32)

# # Path to the input image
# image_path = '/home/qt/callib/Align.jpg'

# # Call the function to create a coordinate system
# create_coordinate_system(image_path)


In [6]:
import cv2
import os
import numpy as np

def create_coordinate_system(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Define the ArUco dictionary
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)

    # Initialize the ArUco detector parameters
    parameters = cv2.aruco.DetectorParameters_create()

    # Detect markers in the image
    corners, ids, rejected = cv2.aruco.detectMarkers(blurred, aruco_dict, parameters=parameters)

    # Draw the detected markers on the image
    image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids)

    # Create a coordinate system image
    coordinate_system = np.zeros_like(image)

    # Check if markers are detected
    if ids is not None and len(ids) >= 4:  # Ensure at least 4 markers are detected
        # Sort the markers based on their IDs
        sorted_indices = np.argsort(ids.flatten())
        sorted_corners = [corners[i] for i in sorted_indices]

        # Iterate over sorted markers
        for i in range(len(sorted_corners)):
            # Draw a coordinate system for each marker
            rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(sorted_corners[i], 0.2, camera_matrix, distortion_coefficients)
            rvec = rvec[0]  # Extract the rotation vector

            # Ensure rvec is a 1x3 vector
            if rvec.shape == (1, 1, 3):
                rvec = rvec[0, 0]

            # Print the coordinates of the marker
            print(f"Marker ID: {ids[i]} - Translation Vector: {tvec[0]} - Rotation Vector: {rvec}")

            # Draw the coordinate system
            coordinate_system = cv2.aruco.drawAxis(
                coordinate_system, 
                np.float32(camera_matrix),  # Ensure camera_matrix is a 3x3 floating-point matrix
                np.float32(distortion_coefficients),  # Ensure distortion_coefficients is a 1x5 floating-point matrix
                rvec=rvec, 
                tvec=tvec[0], 
                length=0.05
            )

            # Draw a box around the coordinate system
            box_size = 100  # Adjust the box size as needed
            cv2.drawMarker(coordinate_system, (int(tvec[0, 0, 0]), int(tvec[0, 0, 1])), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=box_size, thickness=3)

            # Connect to the next marker
            next_index = (i + 1) % len(sorted_corners)
            start_point = tuple(map(int, sorted_corners[i][0][1]))  # Convert to tuple and integer
            end_point = tuple(map(int, sorted_corners[next_index][0][0]))  # Convert to tuple and integer
            cv2.line(coordinate_system, start_point, end_point, (255, 0, 0), 2)

        # Calculate and print the coordinates of the plane
        plane_origin = tuple(map(int, sorted_corners[0][0][0]))  # Use the corner of the first marker as the origin
        plane_x_axis = tuple(map(int, sorted_corners[1][0][1]))  # Use the corner of the second marker as the x-axis
        plane_y_axis = tuple(map(int, sorted_corners[3][0][1]))  # Use the corner of the fourth marker as the y-axis
        
        print(f"Plane Origin: {plane_origin}")
        print(f"Plane X-Axis: {plane_x_axis}")
        print(f"Plane Y-Axis: {plane_y_axis}")

        # Draw a red rectangle to show the boundaries of the x-y plane
        cv2.rectangle(coordinate_system, plane_origin, plane_y_axis, (0, 255, 255), 1)


        # Superimpose the coordinate system on the original detected image
        alpha = 0.3  # Adjust transparency (0.0: fully transparent, 1.0: fully opaque)
        final_image = cv2.addWeighted(image_with_markers, 1 - alpha, coordinate_system, alpha, 0)

        # Save the superimposed image
        cv2.imwrite('/home/qt/callib/imgs/Superimposed_Image.jpg', final_image)

    # Save the images to files
    cv2.imwrite('/home/qt/callib/imgs/Detected_Markers.jpg', image_with_markers)
    cv2.imwrite('/home/qt/callib/imgs/Coordinate_System.jpg', coordinate_system)

# Dummy camera matrix and distortion coefficients
camera_matrix = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]], dtype=np.float32)
distortion_coefficients = np.array([0.1, 0.01, 0, 0, 0], dtype=np.float32)

# Path to the input image
image_path = '/home/qt/visionSort/Align.jpg'

# Call the function to create a coordinate system
create_coordinate_system(image_path)


Marker ID: [2] - Translation Vector: [[-0.17113815  0.04620907  0.80591371]] - Rotation Vector: [[-3.05494988 -0.01506574  0.13369326]]
Marker ID: [3] - Translation Vector: [[0.87130495 0.05225322 0.89373346]] - Rotation Vector: [[-3.10990428 -0.00983012 -0.29839628]]
Marker ID: [1] - Translation Vector: [[-0.15714182  1.5232177   1.04235993]] - Rotation Vector: [[-2.93676032 -0.00665542  0.01068162]]
Marker ID: [0] - Translation Vector: [[0.91849312 1.54668493 1.12297735]] - Rotation Vector: [[-2.96070606  0.02550742 -0.16242899]]
Plane Origin: (117, 198)
Plane X-Axis: (1040, 205)
Plane Y-Axis: (1038, 1260)


'/home/qt/visionSort'